In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes
import gower
from sklearn.metrics import silhouette_score

In [6]:
df = pd.read_csv("Melbourne_housing_FULL.csv")

In [7]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [9]:
df = df.drop(["CouncilArea", "Date", "Method", "SellerG", "YearBuilt", "Address", "Bedroom2", 
             "Propertycount", "Suburb"], axis = 1)

In [10]:
df.describe()

,Rooms,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude
count,34857.000000,2.724700e+04,34856.000000,34856.000000,26631.000000,26129.000000,23047.000000,13742.00000,26881.000000,26881.000000
mean,3.031012,1.050173e+06,11.184929,3116.062859,1.624798,1.728845,593.598993,160.25640,-37.810634,145.001851
std,0.969933,6.414671e+05,6.788892,109.023903,0.724212,1.010771,3398.841946,401.26706,0.090279,0.120169
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.00000,-38.190430,144.423790
25%,2.000000,6.350000e+05,6.400000,3051.000000,1.000000,1.000000,224.000000,102.00000,-37.862950,144.933500
50%,3.000000,8.700000e+05,10.300000,3103.000000,2.000000,2.000000,521.000000,136.00000,-37.807600,145.007800
75%,4.000000,1.295000e+06,14.000000,3156.000000,2.000000,2.000000,670.000000,188.00000,-37.754100,145.071900
max,16.000000,1.120000e+07,48.100000,3978.000000,12.000000,26.000000,433014.000000,44515.00000,-37.390200,145.526350


In [11]:
df = df.dropna(subset=["Price"])
print(df["Price"].isna().sum())

0


In [12]:
df = df.dropna(subset=["Lattitude", "Longtitude"])

In [13]:
Q1 = df["Landsize"].quantile(0.25)
Q3 = df["Landsize"].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR 
df = df[df["Landsize"] <= upper_bound]

In [14]:
lands_median = df["Landsize"].median()
df["Landsize"] = df["Landsize"].fillna(lands_median)

In [15]:
Q1 = df["BuildingArea"].quantile(0.25)
Q3 = df["BuildingArea"].quantile(0.75)
IQR = Q3 - Q1
upper_ba = Q3 + 1.5 * IQR

df = df[df["BuildingArea"].isna() | (df["BuildingArea"] <= upper_ba)]

In [16]:
df["BuildingArea"] = df.groupby("Type")["BuildingArea"].transform(lambda x: x.fillna(x.median()))

In [17]:
for col in ["Car", "Bathroom"]:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val).round().astype(int)

In [ ]:
df.to_csv("Melbourne_housing_CLEANED.csv", index=False)